In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinese-couplets/couplet/vocabs
/kaggle/input/chinese-couplets/couplet/test/out.txt
/kaggle/input/chinese-couplets/couplet/test/in.txt
/kaggle/input/chinese-couplets/couplet/test/.in.txt.swp
/kaggle/input/chinese-couplets/couplet/test/.out.txt.swp
/kaggle/input/chinese-couplets/couplet/train/out.txt
/kaggle/input/chinese-couplets/couplet/train/in.txt


##//作业1：使用中文对联数据集训练带有attention的seq2seq模型，利用tensorboard跟踪。
https://www.kaggle.com/datasets/jiaminggogogo/chinese-couplets* 

In [2]:
import pickle
import torch
import json
from torch.utils.data import DataLoader
from process_seq2seq import get_proc
from seq2seqattencat import Seq2Seq
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm



In [3]:
device = torch.device('cuda')


In [4]:
file_in='/kaggle/input/chinese-couplets/couplet/train/in.txt'
file_out='/kaggle/input/chinese-couplets/couplet/train/out.txt'

In [5]:
###读取数据
try:
    enc_data = []
    dec_data = []

    with open(file_in, 'r', encoding='utf-8') as f1, open(file_out, 'r', encoding='utf-8') as f2:

        for line1, line2 in zip(f1, f2):
            enc = line1.split(" ")
            dec = line2.split(" ")
            
            # 分词
            enc_tks = [element for element in enc if element != "\n"]
            dec_tks = ['BOS'] + list([element for element in dec if element != "\n"]) + ['EOS']
            # 保存
            enc_data.append(enc_tks)
            dec_data.append(dec_tks)

except FileNotFoundError:
    print("文件未找到，请检查文件路径。")
except Exception as e:
    print(f"读取文件时出现错误: {e}")

In [6]:
enc_data[:5]

[['晚', '风', '摇', '树', '树', '还', '挺'],
 ['愿', '景', '天', '成', '无', '墨', '迹'],
 ['丹', '枫', '江', '冷', '人', '初', '去'],
 ['忽',
  '忽',
  '几',
  '晨',
  '昏',
  '，',
  '离',
  '别',
  '间',
  '之',
  '，',
  '疾',
  '病',
  '间',
  '之',
  '，',
  '不',
  '及',
  '终',
  '年',
  '同',
  '静',
  '好'],
 ['闲', '来', '野', '钓', '人', '稀', '处']]

In [7]:
dec_data[:5]

[['BOS', '晨', '露', '润', '花', '花', '更', '红', 'EOS'],
 ['BOS', '万', '方', '乐', '奏', '有', '于', '阗', 'EOS'],
 ['BOS', '绿', '柳', '堤', '新', '燕', '复', '来', 'EOS'],
 ['BOS',
  '茕',
  '茕',
  '小',
  '儿',
  '女',
  '，',
  '孱',
  '羸',
  '若',
  '此',
  '，',
  '娇',
  '憨',
  '若',
  '此',
  '，',
  '更',
  '烦',
  '二',
  '老',
  '费',
  '精',
  '神',
  'EOS'],
 ['BOS', '兴', '起', '高', '歌', '酒', '醉', '中', 'EOS']]

In [8]:
from process_seq2seq import Vocabulary

enc_vocab = Vocabulary.from_documents(enc_data)
dec_vocab = Vocabulary.from_documents(dec_data)

In [9]:
##保存词典
import pickle
with open('vocab.bin','wb') as f:
    pickle.dump((enc_vocab.vocab, dec_vocab.vocab),f)


In [10]:
dec_vocab.vocab['BOS']

7247

In [11]:
dec_vocab.vocab['EOS']

840

In [12]:
from torch.utils.data import DataLoader
from process_seq2seq import get_proc

ds = list(zip(enc_data,dec_data))
dl = DataLoader(ds, batch_size=256, shuffle=True, collate_fn=get_proc(enc_vocab.vocab, dec_vocab.vocab))


In [13]:
len(enc_data)

770491

In [14]:
enc_data[0]

['晚', '风', '摇', '树', '树', '还', '挺']

In [15]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter()

2025-05-07 11:30:13.618704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746617413.799092      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746617413.854896      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
from seq2seqattencat import Seq2Seq

# 构建训练模型
# 模型构建
model = Seq2Seq(
    enc_emb_size=len(enc_vocab.vocab),
    dec_emb_size=len(dec_vocab.vocab),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.to(device)

# 优化器、损失
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
train_loss_cnt=0
# 训练
for epoch in range(20):
    model.train()
    tpbar = tqdm(dl)
    for enc_input, dec_input, targets in tpbar:
        enc_input = enc_input.long()
        dec_input = dec_input.long()
        targets = targets.long()
        enc_input = enc_input.to(device)
        dec_input = dec_input.to(device)
        targets = targets.to(device)

        # 前向传播 
        logits, _ = model(enc_input, dec_input)

        # 计算损失
        # CrossEntropyLoss需要将logits和targets展平
        # logits: [batch_size, seq_len, vocab_size]
        # targets: [batch_size, seq_len]
        # 展平为 [batch_size * seq_len, vocab_size] 和 [batch_size * seq_len]
        loss = criterion(logits.view(-1, logits.size(-1)), targets.view(-1))
        writer.add_scalar('train_loss',loss.item(),train_loss_cnt)
        train_loss_cnt+=1

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tpbar.set_description(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'seq2seq_state.bin')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
Epoch 20, Loss: 1.3863: 100%|██████████| 3010/3010 [02:06<00:00, 23.84it/s]
